# Hypothesis Testing in Healthcare: Drug Safety

A pharmaceutical company GlobalXYZ has just completed a randomized controlled drug trial. To promote transparency and reproducibility of the drug's outcome, they (GlobalXYZ) have presented the dataset to your organization, a non-profit that focuses primarily on drug safety.

The dataset provided contained five adverse effects, demographic data, vital signs, etc. Your organization is primarily interested in the drug's adverse reactions. It wants to know if the adverse reactions, if any, are of significant proportions. It has asked you to explore and answer some questions from the data.

The dataset `drug_safety.csv` was obtained from [Hbiostat](https://hbiostat.org/data/) courtesy of the Vanderbilt University Department of Biostatistics. It contained five adverse effects: headache, abdominal pain, dyspepsia, upper respiratory infection, chronic obstructive airway disease (COAD), demographic data, vital signs, lab measures, etc. The ratio of drug observations to placebo observations is 2 to 1.

For this project, the dataset has been modified to reflect the presence and absence of adverse effects `adverse_effects` and the number of adverse effects in a single individual `num_effects`.

The columns in the modified dataset are:

| Column | Description |
|--------|-------------|
|`sex` | The gender of the individual |
|`age` | The age of the individual |
|`week` | The week of the drug testing |
|`trx` | The treatment (Drug) and control (Placebo) groups |
|`wbc` | The count of white blood cells |
|`rbc` | The count of red blood cells |
|`adverse_effects` | The presence of at least a single adverse effect |
|`num_effects` | The number of adverse effects experienced by a single individual |

The original dataset can be found [here](https://hbiostat.org/data/repo/safety.rda).

Your organization has asked you to explore and answer some questions from the data collected.

**Task 1:** Determine if the proportion of adverse effects differs significantly between the Drug and Placebo groups, saving the p-value as a variable called two_sample_p_value.

**Task 2: **Find out if the number of adverse effects is independent of the treatment and control groups, saving as a variable called num_effects_p_value containing a p-value.

**Task 3:** Examine if there is a significant difference between the ages of the Drug and Placebo groups, storing the p-value of your test in a variable called age_group_effects_p_value.

In [3]:
!pip uninstall scipy -y
!pip uninstall pingouin -y
!pip install pingouin

  Using cached pingouin-0.5.4-py2.py3-none-any.whl (198 kB)
  Using cached scipy-1.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.6 MB)
  Using cached pandas_flavor-0.6.0-py3-none-any.whl (7.2 kB)


In [6]:
# Import packages
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import norm
import pingouin
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
drug_safety = pd.read_csv("drug_safety.csv")

# Start coding here..

# 1. Calculate z-score for two-sample proportion test

    # 1.1 Calculate proportions for adverse effects between placebo and control groups (proportions for placebo and control population are 2:1)
prop_normalized = drug_safety.groupby("trx")["adverse_effects"].value_counts(normalize= True)
p_drug = 0.09546
p_placebo = 0.095238

drug_safety.groupby("trx")["adverse_effects"].count()
n_drug = 0.10727
n_placebo = 0.5376

# Let's set a significance level = 0.5.

    # 1.2 Calculate p_hat
p_hat = (p_drug*n_drug + n_placebo*p_placebo)/ (n_drug + n_placebo)

    # 1.3 Calculate std_error
std_error = np.sqrt(p_hat*(1-p_hat)/n_drug + p_hat*(1-p_hat)/n_placebo)

    # 1.4 Calculate z_score for two tailed test H_0: p_drug - p_placebo = 0
z_score = ((p_drug - p_placebo) - 0)/ std_error

    # Calculate p_value for two tailed test
p_value = 2 * norm.cdf(-abs(z_score))

print(z_score, p_value)

# Using proportions_ztest
prop_unnormalized = drug_safety.groupby("trx")["adverse_effects"].value_counts(normalize= False)
n_affected = np.array([9703, 4864])
n_rows = np.array([9703+1024, 4864+512])

z_score_2, two_sample_p_value = proportions_ztest(n_affected, n_rows, alternative = "two-sided")

print(z_score_2, two_sample_p_value)


0.00022611950699697093 0.9998195827380082
-0.04521826844949136 0.9639333330262497


 p_value is >= 0.964, that means there's strong evidence supporting null hypothesis

 H_0: (p_affected _drug- p_affected _placebo = 0)

 Then, we could argue that the adverse effects do not differ significantly between Drug and Placebo groups.


In [7]:
# Perform chi-square test of indepence
# Let's set 0.1 as significance level

expected, observed, stats = pingouin.chi2_independence(data= drug_safety, x= "trx", y= "num_effects")

print(stats)

num_effects_p_value= 0.615012

                 test    lambda      chi2  dof      pval    cramer     power
0             pearson  1.000000  1.799644  3.0  0.615012  0.010572  0.176275
1        cressie-read  0.666667  1.836006  3.0  0.607131  0.010678  0.179153
2      log-likelihood  0.000000  1.922495  3.0  0.588648  0.010926  0.186033
3       freeman-tukey -0.500000  2.001752  3.0  0.572043  0.011149  0.192379
4  mod-log-likelihood -1.000000  2.096158  3.0  0.552690  0.011409  0.199984
5              neyman -2.000000  2.344303  3.0  0.504087  0.012066  0.220189


/usr/local/lib/python3.10/dist-packages/pingouin/contingency.py:151: UserWarning: Low count on observed frequencies.
  warnings.warn(f"Low count on {name} frequencies.")
/usr/local/lib/python3.10/dist-packages/pingouin/contingency.py:151: UserWarning: Low count on expected frequencies.
  warnings.warn(f"Low count on {name} frequencies.")


Test:
H_0: num_effects is independent of trx
H_1: num_effects is not independent of trx

Significance level: 0.1

pval = 0.615 , which suggest enough evidence supporting H_0.

So we can conclude the number of effects is not related to the suministration of the drug

# Task 3
Examine if there is a significant difference between the ages of the Drug and Placebo groups, storing the p-value of your test in a variable called age_group_effects_p_value.

---
Hypothesis:

H_0 = there isn't significant difference between the ages of the drug and placebo groups

H_1 = the difference of ages between the groups IS significant

Level of significance = 0.01

Plan: perform Wilcoxon-Mann-Whitney for age (continuous numeric variable) between 2 categories (drug and placebo groups)

Result: p_value = 0.256963

In [8]:
# Separate relevant data
group_vs_age = drug_safety[["trx", "age"]]

# Convert data from long to wide format for Wilcoxon
group_vs_age_wide = group_vs_age.pivot(columns = "trx",
                                       values = "age")

pingouin.mwu(x= group_vs_age_wide["Drug"],
             y= group_vs_age_wide["Placebo"],
             alternative = "two-sided")

age_group_effects_p_value = 0.256963

P-value suggest enough evidence suporting H_0. So, we can conclude there isn't significant difference between the ages of the drug and placebo groups.
